### CONFIGURATION

In [1]:
import os
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 200)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [3]:
#VCF
VCF_PATH = "/Users/peterpriestley/hmf/analyses/ensembleRuleTesting/"

# BED
#BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/"
#BED_FILE_NAME = "NA12878_GIAB_highconf_IllFB-IllGATKHC-CG-Ion-Solid_ALLCHROM_v3.2.2_highconf.bed"
BED_PATH = "/Users/peterpriestley/hmf/analyses/slices/"
#BED_FILE_NAME = "cosmicCancerCensus.bed" 
BED_FILE_NAME = "CPCT_Slicing.bed"
USE_BED = True

# HEALTH CHECKS
HEALTH_CSV_PATH="/Users/peterpriestley/hmf/analyses/healthCheckAnalysis/"
HEALTH_CSV_FILENAME="healthCheckFile.csv"

In [46]:
# SPECIFY SAMPLES
VCF_SAMPLES=[]
for x in os.listdir(VCF_PATH):
    if x[-10:] == "sliced.vcf":
        VCF_SAMPLES.append(x)
#VCF_SAMPLES = ['160903_HMFregCPCT_FR10302737_FR10304566_CPCT02070012.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf']

<h3> LOAD VCF

In [5]:
#Load all VCF_SAMPLES to DF
df = pd.DataFrame({})
for VCF_SAMPLE in VCF_SAMPLES:
    VCF_FILE_NAME = VCF_SAMPLE
    SAMPLE_NAMES = {VCF_SAMPLE[40:52]+'R':'normal',VCF_SAMPLE[40:52]+'T':'tumor'}
    if USE_BED:
        bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[40:52],True,True,bed,False)])
    else:
        df = pd.concat([df,aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE[40:52],True)])

reading BED file
Bed File Loaded
reading vcf file: 160903_HMFregCPCT_FR10302737_FR10304566_CPCT02070012.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 67
reading BED file
Bed File Loaded
reading vcf file: 160903_HMFregCPCT_FR10302740_FR10304570_CPCT02050049.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 65
reading BED file
Bed File Loaded
reading vcf file: 160905_HMFregCPCT_FR10301943_FR12251966_CPCT02160001.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 73
reading BED file
Bed File Loaded
reading vcf file: 160905_HMFregCPCT_FR10302743_FR10304571_CPCT02050050.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 57
reading BED file
Bed File Loaded
reading vcf file: 160905_HMFregCPCT_FR12244549_FR12251919_CPCT02160002.filtered_variants_snpEff_snpSift_Cosmicv76_GoNLv5_sliced.vcf
Number variants loaded: 72
reading BED file
Bed File Loaded
reading

In [6]:
#LOAD HEALTH_CHECKS
hc = pd.read_csv(HEALTH_CSV_PATH + HEALTH_CSV_FILENAME)
hc = hc[['SAMPLE','TUMOR_PERCENTAGE','TUMOR_TYPE','PIPELINE_VERSION']]
hc = hc[hc['SAMPLE'].str[12]=="T"]   # only tumor rows
hc['SAMPLE']=hc['SAMPLE'].str[:12]
hc = hc.set_index(['SAMPLE'])
df = pd.merge(df,hc,how='left', left_on='patientName',right_index=True)
df.fillna("Unknown", inplace=True)

<h3> High Level Stats

### LOH variants

In [47]:
pd.pivot_table(df[(df.normalSGT<>"")&(df.normalSGT<>df.tumorSGT)],\
               values='pos', index=['chromPos','patientName','annGene','normalSGT','tumorSGT'], columns=['consensus'], aggfunc='count',margins=True).fillna("")

consensus                                            False True   All
chromPos     patientName  annGene normalSGT tumorSGT                 
10:89720633  CPCT02040035 PTEN    0/1                    1        1.0
             CPCT02050048 PTEN    0/1                    1        1.0
             CPCT02060018 PTEN    0/1                    1        1.0
11:534242    CPCT02010393 HRAS    0/1       1/1               1   1.0
13:32914236  CPCT02020357 BRCA2   0/1                    1        1.0
14:105246325 CPCT02010393 AKT1    0/1       1/1          1        1.0
14:105246407 CPCT02160003 AKT1    0/1       1/1          1        1.0
14:105246565 CPCT02160003 AKT1    0/1                    1        1.0
14:105258892 CPCT02010393 AKT1    0/1                    1        1.0
17:37868189  CPCT02020293 ERBB2   0/1                    1        1.0
17:37884037  CPCT02010393 ERBB2   0/1                    1        1.0
17:41197709  CPCT02020293 BRCA1   0/1                    1        1.0
17:41223094  CPCT02010393 BRCA1   0/1                    1        1.0
             CPCT02160003 BRCA1   0/1                    1        1.0
17:41234470  CPCT02010393 BRCA1   0/1                    1        1.0
17:41244435  CPCT02010393 BRCA1   0/1                    1        1.0
17:41244936  CPCT02010393 BRCA1   0/1                    1        1.0
17:41245237  CPCT02010393 BRCA1   0/1                    1        1.0
17:41245466  CPCT02010393 BRCA1   0/1                    1        1.0
17:41246481  CPCT02010393 BRCA1   0/1       1/1               1   1.0
17:7578645   CPCT02010393 TP53    0/1                    1        1.0
2:212578379  CPCT02010394 ERBB4   0/1       1/1          1        1.0
             CPCT02140015 ERBB4   0/1       1/1          1        1.0
3:10183444   CPCT02020293 VHL     0/1                    1        1.0
4:1795557    CPCT02010325 FGFR3   1/1                    1        1.0
             CPCT02110006 FGFR3   1/1                    1        1.0
             CPCT02160002 FGFR3   1/1                    1        1.0
4:1806012    CPCT02060019 FGFR3   0/1                    1        1.0
4:1808004    CPCT02010375 FGFR3   0/1                    1        1.0
4:1808006    CPCT02010375 FGFR3   0/1                    1        1.0
4:1808010    CPCT02010375 FGFR3   0/1                    1        1.0
All                                                     29    2  31.0

### HIGH and MODERATE impact NON-missense

In [44]:
pd.pivot_table(df[((df['annWorstImpact']=='MODERATE')|(df['annWorstImpact']=='HIGH'))&(~df.annAllEffects.str.contains("missense"))&(df.normalSGT=="")],\
               values='pos', index=['chromPos','patientName','annGene','annWorstEffect','annWorstImpact','ref','normalallele'], columns=['consensus'], aggfunc='count',margins=True).fillna("")

,,,,,,consensus,False,All
chromPos,patientName,annGene,annWorstEffect,annWorstImpact,ref,normalallele,,
10:89692784,CPCT02140013,PTEN,disruptive_inframe_insertion,MODERATE,T,,1.0,1.0
10:89717731,CPCT02060015,PTEN,frameshift_variant,HIGH,T,,1.0,1.0
10:89720741,CPCT02140015,PTEN,stop_gained,HIGH,C,,1.0,1.0
10:89720798,CPCT02160001,PTEN,frameshift_variant,HIGH,GTACT,,1.0,1.0
16:68856020,CPCT02160001,CDH1,stop_gained,HIGH,C,,1.0,1.0
17:7573976,CPCT02060019,TP53,stop_gained,HIGH,T,,1.0,1.0
17:7576851,CPCT02140015,TP53,splice_donor_variant&intron_variant,HIGH,A,,1.0,1.0
17:7576855,CPCT02010397,TP53,stop_gained&splice_region_variant,HIGH,G,,1.0,1.0
17:7577085,CPCT02030261,TP53,stop_gained,HIGH,C,,1.0,1.0


### Missense

In [33]:
pd.pivot_table(df[(df.annAllEffects.str.contains("missense"))&(df.normalSGT=="")],\
               values='pos', index=['chromPos','patientName','annGene','normalSGT','tumorSGT'], columns=['consensus'], aggfunc='count',margins=True).fillna("")

,,,,consensus,False,All
chromPos,patientName,annGene,normalSGT,tumorSGT,,
10:89624305,CPCT02110007,PTEN,,0/1,1.0,1.0
10:89720677,CPCT02060015,PTEN,,0/1,1.0,1.0
12:25398284,CPCT02110006,KRAS,,0/1,1.0,1.0
13:32910968,CPCT02030261,BRCA2,,0/1,1.0,1.0
13:49033926,CPCT02020357,RB1,,0/1,1.0,1.0
16:68842328,CPCT02050048,CDH1,,0/1,1.0,1.0
17:37881000,CPCT02060019,ERBB2,,0/1,1.0,1.0
17:7577536,CPCT02110006,TP53,,0/1,1.0,1.0
17:7577538,CPCT02010390,TP53,,0/1,1.0,1.0


### SUMMARY OF TYPE

In [13]:
pd.pivot_table(df, values='pos', index=['normalSGT',], columns=['tumorSGT'], aggfunc='count').fillna("")

tumorSGT,,0/1,1/1
normalSGT,,,
,,67,7.0
0/1,22,1406,6.0
1/1,3,,1236.0


<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(df):
    if columnName.endswith('allele'):
        ser = df[df[columnName] != ''][columnName[:-6] + 'AF']
        ser = ser.sort_values()
        #ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-6]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller",figsize=[15,6])